**--- Inicio Colab ---**


In [99]:
# Monto drive en colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [117]:
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/hyperparameter_tuning.py' '/content'

In [2]:
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/feature_builder.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/util.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/hyperparameter_tuning.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/feature_builder.py' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/train.csv' '/content'
!cp '/content/gdrive/My Drive/Colab Notebooks/tp2-datos/test.csv' '/content'
!mkdir /content/data/embeddings
!wget -P /content/data/embeddings -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
!gunzip '/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz'
!mv '/content/data/embeddings/GoogleNews-vectors-negative300.bin' '/content/data/embeddings/word2vec.bin'

mkdir: cannot create directory ‘/content/data/embeddings’: File exists
--2020-07-24 08:07:14--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.171.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.171.13|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 1647046227 (1.5G), 1395037602 (1.3G) remaining [application/x-gzip]
Saving to: ‘/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[+++================>]   1.53G  80.8MB/s    in 18s     

2020-07-24 08:07:32 (74.5 MB/s) - ‘/content/data/embeddings/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**--- Fin Colab ---**

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation,Bidirectional,GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from sklearn.metrics import f1_score
import feature_builder
import util
import hyperparameter_tuning
embeddings = None
from hyperopt import hp,STATUS_OK

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [68]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df,encoding_type='mean')
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [69]:
processed, X_processed = prepare_for_cnn(train_df)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

In [71]:
def GRU():
    model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256),input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(256 , activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)])
    return model

In [72]:
model = GRU()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 512)               1368576   
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 257       
Total params: 1,500,161
Trainable params: 1,500,161
Non-trainable params: 0
_________________________________________________________________


In [73]:
history = model.fit(X_train, y_train, epochs=30, shuffle=False, validation_split=0.0)

Epoch 1/30
160/160 [==============================] - 1s 5ms/step - loss: 0.6452 - accuracy: 0.5918
Epoch 2/30
160/160 [==============================] - 1s 5ms/step - loss: 0.5948 - accuracy: 0.6516
Epoch 3/30
160/160 [==============================] - 1s 5ms/step - loss: 0.5555 - accuracy: 0.7020
Epoch 4/30
160/160 [==============================] - 1s 5ms/step - loss: 0.5127 - accuracy: 0.7394
Epoch 5/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4823 - accuracy: 0.7639
Epoch 6/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4577 - accuracy: 0.7939
Epoch 7/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4415 - accuracy: 0.7992
Epoch 8/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4274 - accuracy: 0.8037
Epoch 9/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4194 - accuracy: 0.8106
Epoch 10/30
160/160 [==============================] - 1s 5ms/step - loss: 0.4122 - accuracy: 0.8165

KeyError: ignored

In [74]:
prediction = model.predict_classes(X_test)

In [75]:
accuracy_score(y_test, prediction)

0.8225228810187027

In [76]:
f1 = f1_score(y_test, prediction,average=None)
mean_f1 = (f1[0] + f1[1]) / 2
mean_f1

0.8132142800004267

Para kaggle

In [78]:
y_train = train_df['target'].values

In [79]:
def GRU():
    model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256),input_shape=(X_processed.shape[1], X_processed.shape[2])),
    tf.keras.layers.Dense(256 , activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)])
    return model

In [80]:
model = GRU()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 512)               1368576   
_________________________________________________________________
dense_20 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 257       
Total params: 1,500,161
Trainable params: 1,500,161
Non-trainable params: 0
_________________________________________________________________


In [81]:
history = model.fit(X_processed, y_train, epochs=30, shuffle=False, validation_split=0.0)

Epoch 1/30
238/238 [==============================] - 1s 5ms/step - loss: 0.6645 - accuracy: 0.5918
Epoch 2/30
238/238 [==============================] - 1s 5ms/step - loss: 0.6152 - accuracy: 0.6386
Epoch 3/30
238/238 [==============================] - 1s 5ms/step - loss: 0.5751 - accuracy: 0.6750
Epoch 4/30
238/238 [==============================] - 1s 5ms/step - loss: 0.5406 - accuracy: 0.7136
Epoch 5/30
238/238 [==============================] - 1s 5ms/step - loss: 0.5077 - accuracy: 0.7453
Epoch 6/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4763 - accuracy: 0.7710
Epoch 7/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4566 - accuracy: 0.7869
Epoch 8/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4397 - accuracy: 0.7986
Epoch 9/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4290 - accuracy: 0.8024
Epoch 10/30
238/238 [==============================] - 1s 5ms/step - loss: 0.4188 - accuracy: 0.8063

KeyError: ignored

In [82]:
ids = test_df['id']

processed, final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

Embeddings loaded!
Percentage of words covered in the embeddings = 0.5707598689343111
Embeddings loaded!
Percentage of words covered in the embeddings = 0.6603057058767683


,target
id,
0,1
2,1
3,1
9,1
11,1
...,...
10861,0
10865,1
10868,1


In [83]:
final_df['target'].value_counts()

0    2204
1    1059
Name: target, dtype: int64

In [84]:
final_df.to_csv('gru.csv')

Hyperopt (bayesian)

In [85]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
Y = train_df['target'].values

In [86]:
processed = feature_builder.process_dataset(train_df)
X_processed = processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1])
X_train, X_test, y_train, y_test = train_test_split(X_processed,Y,test_size = 0.33,random_state = 17)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.4937444933920705
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5879980378263476


In [176]:
def gru(params):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(params['units']),input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(tf.keras.layers.Dense(params['units2'], activation=params['activation']))
    model.add(tf.keras.layers.Dropout(params['dropout']))
    model.add(tf.keras.layers.Dense(1))

    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(params['lr']),
              metrics=['accuracy'])
    
    result = model.fit(X_train, y_train, epochs=50, shuffle=True, validation_split=0.2,verbose=0)
    prediction = model.predict_classes(X_test)
    f1 = f1_score(y_test, prediction,average=None)
    mean_f1 = (f1[0] + f1[1]) / 2
    mean_f1
    print('\nScore:',mean_f1)
    return {'loss': -mean_f1, 'status': STATUS_OK, 'model': model}


In [177]:
space = {'units': hp.choice('units',[16,32,64,128,256]),
        'units2': hp.choice('units2',[16,32,64,128,256]),
        'dropout': hp.uniform('dropout',0,1),
        'activation': hp.choice('activation',['relu','sigmoid']),
        'lr': hp.uniform('lr',1e-4,1e-1)}

In [ ]:
import importlib
importlib.reload(hyperparameter_tuning)
hyperparameter_tuning.bayesian_optimization(gru,space,max_evals=100)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]